# Create tables in Redshift DB and load data


In [ ]:
!pip install psycopg2-binary

### Import Dependencies

In [ ]:
import boto3
import json
import sagemaker
import os
import psycopg2

sagemaker_session = sagemaker.Session()
s3_client = boto3.client("s3")
default_bucket = sagemaker_session.default_bucket() 
execution_role = sagemaker.get_execution_role()
secretId = "SecretForRedshiftSageMakerDemo2023"
local_data_folder = "./data"
tgt_data_s3_folder = "InputData"

#### Upload local data files to S3

In [ ]:
for file in os.listdir(local_data_folder):
    s3_client.upload_file(f"{local_data_folder}/{file}", f"{default_bucket}",f"{tgt_data_s3_folder}/{file}")
    print(f"Uploaded {local_data_folder}/{file} to s3://{default_bucket}/{tgt_data_s3_folder}/{file}")

#### Retrieve redshift connection details from Secrets Manager

In [ ]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId=secretId)
database_secrets = json.loads(response['SecretString'])
username = database_secrets['username']
password = database_secrets['password']
host = database_secrets['host']
port = database_secrets['port']
engine = database_secrets['engine']
#print("jdbc:" + engine + "://" + host + ":" + str(port) + "/dev")

In [ ]:
conn=psycopg2.connect(
    dbname   = "dev", 
    host     = host, 
    port     = port, 
    user     = username, 
    password = password
)
conn.autocommit = True
cursor = conn.cursor()

Uncomment and run below code.
_**This step is only required if you want to drop sagemakerdemo schema and all the tables under the schema._** For the first run it's not required.

In [ ]:
#cursor.execute(open("sql/drop-schema.sql", "r").read())

#### Create sagemakerdemo schema and tables

In [ ]:
cursor.execute(open("sql/create-schema.sql", "r").read())
cursor.execute(open("sql/create-tables.sql", "r").read())

In [ ]:
def load_input_data(table_name, file_name, iam_role,cursor):
    input_data_file = f"s3://{default_bucket}/InputData/{file_name}"
    statement = """
      copy {}
      from '{}'
      iam_role '{}'
      csv
      ignoreheader 1
      timeformat 'YYYY-MM-DDTHH:MI:SS'
    """.format(table_name, input_data_file, iam_role)
    #print(statement)
    cursor.execute(statement)
    print(f"{table_name} - Data inserted successfully")
 

#### Load csv data into Redshift tables

This can take 1 or 2 minutes to load the csv data into Redshift tables.

In [ ]:
load_input_data('sagemakerdemo.dim_user','dim_user_synthetic.csv' ,execution_role,cursor)
load_input_data('sagemakerdemo.dim_place','dim_place_synthetic.csv' ,execution_role,cursor)
load_input_data('sagemakerdemo.fact_rating','fact_rating_synthetic.csv' ,execution_role,cursor)

#### Verify data load in Redshift tables

In [ ]:
sql  =  '''select 'dim_user' as Tablename, count(*) as counts from sagemakerdemo.dim_user 
            union
            select 'dim_place',count(*)  from sagemakerdemo.dim_place 
            union
            select 'fact_rating', count(*)  from sagemakerdemo.fact_rating 
            order by 2;'''
cursor.execute(sql)
print(cursor.fetchall())


#### Close the connection

In [ ]:
conn.close()